In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
import datetime as dt

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas.core import datetools

from sklearn.preprocessing import LabelEncoder

import numpy as np

import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [6]:
# df_raw = pd.read_excel('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HealthMart_Performance_Data_clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c')
df_raw = pd.read_excel('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HM_Master_Data_Analytics_Clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c')
# data_path = 'c:/users/elkzze1/work/ds201_capstone_data/HM_Master_Data_Analytics_Clean.xlsx' 
# df_raw = pd.read_excel('C:/Users/ekvtjdx/MacroHelix/data/HM_Master_Data_Analytics_Clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c', index_col=0)
df_raw.head()

,Account,Nabp_Ncpdp,Npi_Num,Dea_Num,Target,Store_Name,Address-Delivery,City-Delivery,ST,Zip,...,Profitability,Annual_Dispense_Fee_Total,Macro_PID,Script_Count,Capture_Rate,Specialty_Spend,Specialty_Product_Purchases,Generic,Brand,Generic_Ratio
0,10491,3800047,1205901485,AM1635474,10491.0,POSTAL PHARMACY,47809 HIGHWAY 58,OAKRIDGE,OR,97463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17475,3802421,1912059569,AH1621336,17475.0,HOWARDS DRUGS,101 NORTH F STREET,LAKEVIEW,OR,97630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19901,3804425,1114456563,AK1605887,19901.0,HARVARD AVENUE DRUGS,1175 HARVARD AVE,ROSEBURG,OR,97471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,309745,3804324,1437294808,FS5689964,309745.0,REEDSPORT PHARMACY,1409 HWY 101,REEDSPORT,OR,97467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,652229,3844366,1275808743,FJ3131290,652229.0,PHILOMATH FAMILY PHARMACY,1640 MAIN STREET,PHILOMATH,OR,97370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_raw.columns

Index(['Nabp_Ncpdp', 'Npi_Num', 'Dea_Num', 'Target', 'Store_Name',
       'Address-Delivery', 'City-Delivery', 'ST', 'Zip', 'Owner_Mgr_Full_Name',
       'Owner_Email', 'Phone_Number', 'Hm_Start_Dt', 'Channel_Type_Cd',
       'Cust_Type_Desc', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded',
       'Buying_Group', 'Chain_Name', 'Sub_Group_Name', 'Region_Name',
       'Region_Director', 'Pharm_D', 'Vpgm_Name', 'Vps_Name', 'Dsm_Name',
       'RSM_Name', 'Salesperson', 'Salesperson_Group', 'AH_Program',
       'Auto_Ship_Ind', 'DLC_Program', 'FEM_Program', 'HM_Circular_Program',
       'Internal_Decor', 'Interior_Decor_Signage_Available_Funds',
       'LMS_Enrolled', 'LMS_Available_Matching_Funds',
       'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuIPP_Login', 

In [118]:
# unnamed_cols = df_clean.select(lambda col: col.startswith('Unnamed'), axis=1)
# unnamed_cols.columns

In [262]:
# df_raw = df_raw.drop(['Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88', 'Unnamed: 89'], axis=1)

In [5]:
df_raw.Internal_Decor.unique()

array(['X', nan], dtype=object)

In [7]:
df_clean = df_raw

In [ ]:
# def fill_in_missing_dates(df, date_col_name = 'date',date_order = 'asc', fill_value = 0, days_back = 30):

#     df.set_index(date_col_name,drop=True,inplace=True)
#     df.index = pd.DatetimeIndex(df.index)
#     d = datetime.now().date()
#     d2 = d - timedelta(days = days_back)
#     idx = pd.date_range(d2, d, freq = "D")
#     df = df.reindex(idx,fill_value=fill_value)
#     df[date_col_name] = pd.DatetimeIndex(df.index)

#     return df

In [8]:
# if you want to operate on multiple columns, put them in a list like so:
dollar_cols = ['Interior_Decor_Signage_Available_Funds', 'LMS_Available_Matching_Funds', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt', 
               'Gnrc_Rx_Sls_Amt','Os_Gnrc_Rx_Sls_Amt','Otc_Sls_Amt', 'LMS_Enrolled', 'LMS_Available_Matching_Funds', 
               'Vaccine_Items_Sls_Amt']
X_cols = ['Internal_Decor', 'Open_Smart', 'PIP_Program', 'PQS_Program', 'SCS_Program', 'Specialty_Solutions', 'Vaccine_Items',
          'Vaccine_Starter', 'YPO']
y_n_cols = ['340B_Active', 'LMS_Enrolled', 'LMS_Used_Funds_Most_Recent_3_Months', 'LMS_Used_Funds_Previous_3_Months']
yes_cols = ['Auto_Ship_Ind', 'Spar_Coverage_Ind']
empt_num_cols = ['Profitability', 'Annual_Dispense_Fee_Total', 'Script_Count', 'Macro_PID', 'Capture_Rate', 'Specialty_Spend',
                'Specialty_Product_Purchases', 'Generic', 'Brand', 'Generic_Ratio', 'Vaccine_Items_Count']
date_cols = ['Hm_Start_Dt','KYN_Training', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date'] 
df_clean[X_cols] = df_clean[X_cols].applymap(lambda x: 1 if x == 'X' else 0)
df_clean[y_n_cols] = df_clean[y_n_cols].applymap(lambda x: 1 if x == 'Y' else 0)
df_clean[yes_cols] = df_clean[yes_cols].applymap(lambda x: 1 if x == 'Yes' else 0)
df_clean['3rd_Party_Vendor'] = df_clean['3rd_Party_Vendor'].apply(lambda x: 'UNK' if pd.isnull(x) else x).astype(str)
df_clean['Hospital_Associated'] = df_clean['Hospital_Associated'].apply(lambda x: 'UNK' if pd.isnull(x) else x).astype(str)
df_clean['IS_HID'] = df_clean['HID'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_clean[empt_num_cols] = df_clean[empt_num_cols].applymap(lambda x: 0 if pd.isnull(x) else x)
df_clean['DLC_Program'] = df_clean['DLC_Program'].apply(lambda x: 'DLC_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean['FEM_Program'] = df_clean['FEM_Program'].apply(lambda x: 'FEM_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean['HM_Circular_Program'] = df_clean['HM_Circular_Program'].apply(lambda x: 'HM_Circular_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean[dollar_cols]=df_clean[dollar_cols].fillna(0)
df_clean[date_cols]=df_clean[date_cols].fillna(pd.Timestamp('19740101'))
df_clean['Target']=df_clean['Target'].fillna(0)


In [8]:
# df_clean['Capture_Rate'].unique
# df_clean['Capture_Rate'].value_counts()
# df_clean['DLC Program'].isnull().values.any()
# df_raw['3rd Party Vendor'] = df_raw['3rd Party Vendor'].astype(str)
# tmp = df_raw['3rd Party Vendor'].fillna(''); isEmpty = tmp==''
# tmp

In [9]:
# df_raw['3rd Party Vendor'].dtype
df_clean.head(5)

,Nabp_Ncpdp,Npi_Num,Dea_Num,Target,Store_Name,Address-Delivery,City-Delivery,ST,Zip,Owner_Mgr_Full_Name,...,Profitability,Annual_Dispense_Fee_Total,Macro_PID,Script_Count,Capture_Rate,Specialty_Spend,Specialty_Product_Purchases,Generic,Brand,Generic_Ratio
Account,,,,,,,,,,,,,,,,,,,,,
10491,3800047,1205901485,AM1635474,10491.0,POSTAL PHARMACY,47809 HIGHWAY 58,OAKRIDGE,OR,97463,Laurie Patty,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17475,3802421,1912059569,AH1621336,17475.0,HOWARDS DRUGS,101 NORTH F STREET,LAKEVIEW,OR,97630,Jeff Howard,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19901,3804425,1114456563,AK1605887,19901.0,HARVARD AVENUE DRUGS,1175 HARVARD AVE,ROSEBURG,OR,97471,Philip Kaser,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309745,3804324,1437294808,FS5689964,309745.0,REEDSPORT PHARMACY,1409 HWY 101,REEDSPORT,OR,97467,JAMAICA STANDIFORD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
652229,3844366,1275808743,FJ3131290,652229.0,PHILOMATH FAMILY PHARMACY,1640 MAIN STREET,PHILOMATH,OR,97370,SETH PHAN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# df_clean = df_raw[['Account','Store Name','AddressDelivery','CityDelivery','ST','Zip','Region','DC Name','PSAO Expanded',
#                    'Buying Group','AH Program','Auto Ship Ind','DLC Program','FEM Program','HM Circular Program',
#                    'Internal Decor','Interior Decor / Signage Available Funds','LMS Enrolled','LMS Available Matching Funds',
#                    'LMS Used Funds Most Recent 3 Months','LMS Used Funds Previous 3 Months','Open Smart',
#                    'MRA Program Type  Active AH','OTC Frontend Size','PIP Program','Pog Code Name', 'PQS Enrolled',
#                    'Last EQuIPP Login','SCS Program','Specialty Solutions','Vaccine Items','Vaccine Starter',
#                    'Vaccine Items Count','Vaccine Items Sls Amt','YPO','Tot Sls Amt','Brand Rx Sls Amt','Gnrc Rx Sls Amt',
#                    'Os Gnrc Rx Sls Amt','Otc Sls Amt','3rd Party Vendor','Hospital Associated','340B Active']]
# df_clean.head()

In [10]:
# df_clean['LMS_Enrolled'].value_counts()
df_clean['LMS_Enrolled'].unique()

array([1, 0], dtype=int64)

In [11]:
# df_clean['Interior Decor / Signage Available Funds'].value_counts

In [12]:
df_clean.columns

Index(['Nabp_Ncpdp', 'Npi_Num', 'Dea_Num', 'Target', 'Store_Name',
       'Address-Delivery', 'City-Delivery', 'ST', 'Zip', 'Owner_Mgr_Full_Name',
       'Owner_Email', 'Phone_Number', 'Hm_Start_Dt', 'Channel_Type_Cd',
       'Cust_Type_Desc', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded',
       'Buying_Group', 'Chain_Name', 'Sub_Group_Name', 'Region_Name',
       'Region_Director', 'Pharm_D', 'Vpgm_Name', 'Vps_Name', 'Dsm_Name',
       'RSM_Name', 'Salesperson', 'Salesperson_Group', 'AH_Program',
       'Auto_Ship_Ind', 'DLC_Program', 'FEM_Program', 'HM_Circular_Program',
       'Internal_Decor', 'Interior_Decor_Signage_Available_Funds',
       'LMS_Enrolled', 'LMS_Available_Matching_Funds',
       'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuIPP_Login', 

In [9]:
# to drop from df_clean
drop_cols = ['Nabp_Ncpdp','Npi_Num','Dea_Num','Target','Store_Name','Address-Delivery','City-Delivery','Cust_Type_Desc', 
             'Owner_Mgr_Full_Name', 'Owner_Email', 'Phone_Number','Sub_Group_Name','Region_Director','Pharm_D','Vpgm_Name',
             'Vps_Name','Dsm_Name', 'RSM_Name','Salesperson_Group', 'Zip_of_HID','HID','Zip_Match_Level','340B_ID']
df_clean = df_clean.drop(drop_cols, axis=1)


In [14]:
# df_clean = df_clean[['ST','Zip','Hm_Start_Dt','Channel_Type_Cd','Cust_Type_Desc','Bus_Type_Desc','Region','DC_Name',
#                      'PSAO_Expanded','Buying_Group','Chain_Name','Region_Name','Salesperson','AH_Program','Auto_Ship_Ind',
#                      'DLC_Program','FEM_Program','HM_Circular_Program','Internal_Decor','Interior_Decor_Signage_Available_Funds',
#                      'LMS_Enrolled','LMS_Available_Matching_Funds','LMS_Used_Funds_Most_Recent_3_Months',
#                      'LMS_Used_Funds_Previous_3_Months','Open_Smart','MRA_Program_Type_-_Active_AH','Msa_Dma',
#                      'OTC_Frontend_Size','PIP_Program','Pog_Code_Name','KYN_Training','PQS_Enrolled','PQS_Program',
#                      'Last_EQuIPP_Login','PQS_Enrolled_Date','SCS_Program','Signage_Program','Spar_Coverage_Ind',
#                      'Specialty_Solutions','Vaccine_Items','Vaccine_Starter','Vaccine_Items_Count','Vaccine_Items_Sls_Amt','YPO',
#                      'Tot_Sls_Amt','Brand_Rx_Sls_Amt','Gnrc_Rx_Sls_Amt','Os_Gnrc_Rx_Sls_Amt','Otc_Sls_Amt','HID',
#                      '3rd_Party_Vendor','Hospital_Associated','340B_Active','Profitability','Annual_Dispense_Fee_Total',
#                      'Macro_PID','Script_Count','Capture_Rate','Specialty_Spend','Specialty_Product_Purchases','Generic',
#                      'Brand','Generic_Ratio']]

In [15]:
df_clean.columns

Index(['ST', 'Zip', 'Hm_Start_Dt', 'Channel_Type_Cd', 'Bus_Type_Desc',
       'Region', 'DC_Name', 'PSAO_Expanded', 'Buying_Group', 'Chain_Name',
       'Region_Name', 'Salesperson', 'AH_Program', 'Auto_Ship_Ind',
       'DLC_Program', 'FEM_Program', 'HM_Circular_Program', 'Internal_Decor',
       'Interior_Decor_Signage_Available_Funds', 'LMS_Enrolled',
       'LMS_Available_Matching_Funds', 'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date', 'SCS_Program',
       'Signage_Program', 'Spar_Coverage_Ind', 'Specialty_Solutions',
       'Vaccine_Items', 'Vaccine_Starter', 'Vaccine_Items_Count',
       'Vaccine_Items_Sls_Amt', 'YPO', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt',
       'Gnrc_Rx_Sls_Amt', 'Os_Gnrc_Rx_Sls_Amt', 'Otc_Sls_Amt', 'IS_HID',
   

In [10]:
df_clean[date_cols]=df_clean[date_cols].applymap(dt.datetime.toordinal)
# these cat_cols need to have categorized data so we can include them in the analysis
cat_cols = ['DLC_Program', 'FEM_Program', 'HM_Circular_Program', 'PQS_Enrolled', 'ST', 'Pog_Code_Name',
            'DC_Name', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded', 'Buying_Group', 'Chain_Name', 'Region_Name', 
            'Salesperson', 'AH_Program', 'OTC_Front-end_Size', 'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'Signage_Program', 
            '3rd_Party_Vendor', 'Hospital_Associated']
le = LabelEncoder()
# df_clean['ST_encoded'] = le.fit_transform(df_clean['ST'])
# df_clean['DLC_Program_encoded'] = le.fit_transform(df_clean['DLC_Program'])
for lbl in cat_cols:
    new_la = lbl + '_encoded'
    df_clean[new_la] = le.fit_transform(df_clean[lbl].astype(str))
# drop_enc_cols = (['DLC_Program', 'FEM_Program', 'HM_Circular_Program', 'Internal_Decor', 'PQS_Enrolled', 'ST', 'Pog_Code_Name',
#                     'DC_Name', 'Bus_Type_Desc', 'Region', 'PSAO_Expanded', ])
df_clean = df_clean.drop(cat_cols, axis=1)
df_clean.head(5)

,Account,Zip,Hm_Start_Dt,Channel_Type_Cd,Auto_Ship_Ind,Internal_Decor,Interior_Decor_Signage_Available_Funds,LMS_Enrolled,LMS_Available_Matching_Funds,LMS_Used_Funds_Most_Recent_3_Months,...,Chain_Name_encoded,Region_Name_encoded,Salesperson_encoded,AH_Program_encoded,OTC_Front-end_Size_encoded,MRA_Program_Type_-_Active_AH_encoded,Msa_Dma_encoded,Signage_Program_encoded,3rd_Party_Vendor_encoded,Hospital_Associated_encoded
0,10491,97463,732524,30,1,1,1724.0,1,822.0,1,...,39,23,1,4,1,7,183,0,18,15
1,17475,97630,732687,30,1,1,0.0,1,0.0,1,...,39,23,1,0,3,3,190,0,18,15
2,19901,97471,732616,30,0,1,0.0,1,0.0,1,...,39,23,1,0,4,2,183,0,18,15
3,309745,97467,735998,30,0,0,2500.0,1,0.0,0,...,39,23,1,0,1,8,183,3,18,15
4,652229,97370,734702,30,0,1,0.0,1,340.0,1,...,38,23,1,0,0,8,183,0,18,15


In [17]:
# label_encoder=LabelEncoder()
# X['DLC Program'] = labelencoder_X.fit_transform(X['DLC Program'])
# df["ModelNumeric"] = df["Model"].astype("category", ordered = True).cat.codes # Thank you B.Turnwald

In [18]:
# df_clean['Target'].unique()
# df_clean.isnull().head(6)
# df_clean.isnull().values.any()

null_columns=df_clean.columns[df_clean.isnull().any()]
df_clean[null_columns].isnull().sum()


Series([], dtype: float64)

In [19]:
# print(df_raw.loc[df_raw['340B Active'] == 1])
# df_is = df_clean.loc[df_clean['340B Active'] == 1]
# df_is.head()

In [20]:
df_clean.head()

,Zip,Hm_Start_Dt,Channel_Type_Cd,Auto_Ship_Ind,Internal_Decor,Interior_Decor_Signage_Available_Funds,LMS_Enrolled,LMS_Available_Matching_Funds,LMS_Used_Funds_Most_Recent_3_Months,LMS_Used_Funds_Previous_3_Months,...,Chain_Name_encoded,Region_Name_encoded,Salesperson_encoded,AH_Program_encoded,OTC_Front-end_Size_encoded,MRA_Program_Type_-_Active_AH_encoded,Msa_Dma_encoded,Signage_Program_encoded,3rd_Party_Vendor_encoded,Hospital_Associated_encoded
Account,,,,,,,,,,,,,,,,,,,,,
10491,97463,732524,30,1,1,1724.0,1,822.0,1,1,...,39,23,1,4,1,7,183,0,18,15
17475,97630,732687,30,1,1,0.0,1,0.0,1,1,...,39,23,1,0,3,3,190,0,18,15
19901,97471,732616,30,0,1,0.0,1,0.0,1,1,...,39,23,1,0,4,2,183,0,18,15
309745,97467,735998,30,0,0,2500.0,1,0.0,0,0,...,39,23,1,0,1,8,183,3,18,15
652229,97370,734702,30,0,1,0.0,1,340.0,1,0,...,38,23,1,0,0,8,183,0,18,15


In [21]:
df_clean.columns

Index(['Zip', 'Hm_Start_Dt', 'Channel_Type_Cd', 'Auto_Ship_Ind',
       'Internal_Decor', 'Interior_Decor_Signage_Available_Funds',
       'LMS_Enrolled', 'LMS_Available_Matching_Funds',
       'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart', 'PIP_Program',
       'KYN_Training', 'PQS_Program', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date',
       'SCS_Program', 'Spar_Coverage_Ind', 'Specialty_Solutions',
       'Vaccine_Items', 'Vaccine_Starter', 'Vaccine_Items_Count',
       'Vaccine_Items_Sls_Amt', 'YPO', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt',
       'Gnrc_Rx_Sls_Amt', 'Os_Gnrc_Rx_Sls_Amt', 'Otc_Sls_Amt', 'IS_HID',
       '340B_Active', 'Profitability', 'Annual_Dispense_Fee_Total',
       'Macro_PID', 'Script_Count', 'Capture_Rate', 'Specialty_Spend',
       'Specialty_Product_Purchases', 'Generic', 'Brand', 'Generic_Ratio',
       'DLC_Program_encoded', 'FEM_Program_encoded',
       'HM_Circular_Program_encoded', 'PQS_Enrolled_encoded', 'ST_enc

In [22]:
# df_clean.loc[:, df_clean.dtypes == object]
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4846 entries, 10491 to 994517
Data columns (total 62 columns):
Zip                                       4846 non-null int64
Hm_Start_Dt                               4846 non-null int64
Channel_Type_Cd                           4846 non-null int64
Auto_Ship_Ind                             4846 non-null int64
Internal_Decor                            4846 non-null int64
Interior_Decor_Signage_Available_Funds    4846 non-null float64
LMS_Enrolled                              4846 non-null int64
LMS_Available_Matching_Funds              4846 non-null float64
LMS_Used_Funds_Most_Recent_3_Months       4846 non-null int64
LMS_Used_Funds_Previous_3_Months          4846 non-null int64
Open_Smart                                4846 non-null int64
PIP_Program                               4846 non-null int64
KYN_Training                              4846 non-null int64
PQS_Program                               4846 non-null int64
Last_EQuIPP_Log

In [28]:
df_clean['IS_HID'].unique()
df_clean['IS_HID'].value_counts()

0    4669
1     177
Name: IS_HID, dtype: int64

In [11]:
X = df_clean[[
       'Zip', 'Hm_Start_Dt', 'Channel_Type_Cd', 'Auto_Ship_Ind',
       'Interior_Decor_Signage_Available_Funds', 'LMS_Enrolled',
       'LMS_Available_Matching_Funds', 'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart', 'PIP_Program',
       'KYN_Training', 'PQS_Program', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date',
       'SCS_Program', 'Spar_Coverage_Ind', 'Specialty_Solutions',
       'Vaccine_Items', 'Vaccine_Starter', 'Vaccine_Items_Count',
       'Vaccine_Items_Sls_Amt', 'YPO', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt',
       'Gnrc_Rx_Sls_Amt', 'Os_Gnrc_Rx_Sls_Amt', 'Otc_Sls_Amt', 
       '340B_Active', 'Profitability', 'Annual_Dispense_Fee_Total',
       'Macro_PID', 'Script_Count', 'Capture_Rate', 'Specialty_Spend',
       'Specialty_Product_Purchases', 'Generic', 'Brand', 'Generic_Ratio',
       'DLC_Program_encoded', 'FEM_Program_encoded',
       'HM_Circular_Program_encoded', 'Internal_Decor',
       'PQS_Enrolled_encoded', 'ST_encoded', 'Pog_Code_Name_encoded',
       'DC_Name_encoded', 'Bus_Type_Desc_encoded', 'Region_encoded',
       'PSAO_Expanded_encoded', 'Buying_Group_encoded', 'Chain_Name_encoded',
       'Region_Name_encoded', 'Salesperson_encoded', 'AH_Program_encoded',
       'OTC_Front-end_Size_encoded', 'MRA_Program_Type_-_Active_AH_encoded',
       'Msa_Dma_encoded', 'Signage_Program_encoded',
       '3rd_Party_Vendor_encoded', 'Hospital_Associated_encoded']]

y = df_clean['IS_HID'] 
# X = sm.add_constant(X)
# lr = sm.OLS(y, X)
# lr = lr.fit()
# lr.summary()  
logit = sm.Logit(y, X)
result = logit.fit(method='bfgs') 
result.summary()
# model = LogisticRegression(fit_intercept=False)
# mdl = model.fit(X,y)
# model.coef_


C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWa

         Current function value: 0.693147
         Iterations: 0
         Function evaluations: 16
         Gradient evaluations: 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 IS_HID   No. Observations:                 4846
Model:                          Logit   Df Residuals:                     4785
Method:                           MLE   Df Model:                           60
Date:                Sun, 11 Feb 2018   Pseudo R-squ.:                  -3.422
Time:                        09:31:20   Log-Likelihood:                -3359.0
converged:                      False   LL-Null:                       -759.55
                                        LLR p-value:                     1.000
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Zip                                             0   2.12e-06          0      1.000   -4.16e-06    4.16e-06
Hm_Start_Dt                                     0   1.62e-05          0      1.000   -3.17e-05    3.17e-05
Channel_Type_Cd                                 0      0.028          0      1.000      -0.055       0.055
Auto_Ship_Ind                                   0      0.068          0      1.000      -0.133       0.133
Interior_Decor_Signage_Available_Funds          0   3.77e-05          0      1.000   -7.38e-05    7.38e-05
LMS_Enrolled                                    0      0.090          0      1.000      -0.176       0.176
LMS_Available_Matching_Funds                    0   1.43e-05          0      1.000   -2.81e-05    2.81e-05
LMS_Used_Funds_Most_Recent_3_Months             0      0.092          0      1.000      -0.180       0.180
LMS_Used_Funds_Previous_3_Months                0      0.092          0      1.000      -0.181       0.181
Open_Smart                                      0      0.118          0      1.000      -0.230       0.230
PIP_Program                                     0      0.079          0      1.000      -0.155       0.155
KYN_Training                                    0   4.91e-06          0      1.000   -9.63e-06    9.63e-06
PQS_Program                                     0      0.110          0      1.000      -0.215       0.215
Last_EQuIPP_Login                               0   5.63e-06          0      1.000    -1.1e-05     1.1e-05
PQS_Enrolled_Date                               0   1.58e-05          0      1.000    -3.1e-05     3.1e-05
SCS_Program                                     0      0.078          0      1.000      -0.152       0.152
Spar_Coverage_Ind                               0      0.079          0      1.000      -0.154       0.154
Specialty_Solutions                             0      0.165          0      1.000      -0.323       0.323
Vaccine_Items                                   0      0.083          0      1.000      -0.162       0.162
Vaccine_Starter                                 0      0.139          0      1.000      -0.272       0.272
Vaccine_Items_Count                             0      0.026          0      1.000      -0.052       0.052
Vaccine_Items_Sls_Amt                           0   3.14e-05          0      1.000   -6.16e-05    6.16e-05
YPO                                             0      0.080          0      1.000      -0.157       0.157
Tot_Sls_Amt                                     0      0.045          0      1.000      -0.087       0.087
Brand_Rx_Sls_Amt                                0      0.045          0      1.000      -0.087       0.087
Gnrc_Rx_Sls_Amt                                 0      0.045          0      1.000      -0.087       0.087
Os_Gnrc_Rx_Sls_Amt                              0      0.045          0      1.000      -0.087       0.087
Otc_Sls_Amt                    

In [83]:
X.head(10)

,Zip,Hm_Start_Dt,Channel_Type_Cd,Auto_Ship_Ind,Interior_Decor_Signage_Available_Funds,LMS_Enrolled,LMS_Available_Matching_Funds,LMS_Used_Funds_Most_Recent_3_Months,LMS_Used_Funds_Previous_3_Months,Open_Smart,...,Chain_Name_encoded,Region_Name_encoded,Salesperson_encoded,AH_Program_encoded,OTC_Front-end_Size_encoded,MRA_Program_Type_-_Active_AH_encoded,Msa_Dma_encoded,Signage_Program_encoded,3rd_Party_Vendor_encoded,Hospital_Associated_encoded
Account,,,,,,,,,,,,,,,,,,,,,
10491,97463,732524,30,1,1724.0,1,822.0,1,1,0,...,39,23,1,4,1,7,183,0,18,15
17475,97630,732687,30,1,0.0,1,0.0,1,1,0,...,39,23,1,0,3,3,190,0,18,15
19901,97471,732616,30,0,0.0,1,0.0,1,1,0,...,39,23,1,0,4,2,183,0,18,15
309745,97467,735998,30,0,2500.0,1,0.0,0,0,0,...,39,23,1,0,1,8,183,3,18,15
652229,97370,734702,30,0,0.0,1,340.0,1,0,0,...,38,23,1,0,0,8,183,0,18,15
121439,49242,735369,30,1,2500.0,1,2415.0,1,1,0,...,22,23,2,0,4,2,51,1,18,10
9948,43606,734825,30,1,0.0,1,2233.0,1,1,0,...,36,40,2,0,1,3,48,1,18,15
13973,48101,735510,30,0,2500.0,1,1200.0,0,0,0,...,36,40,2,0,6,8,6,2,18,15
18052,48033,735206,30,0,0.0,1,1200.0,0,0,0,...,36,40,2,0,0,8,6,0,18,15


In [84]:
for i, col in enumerate(X.columns):            # check VIF
    print('VIF {}: {}'.format(col,variance_inflation_factor(X.values,i)))

VIF Zip: 17.626985413625853
VIF Hm_Start_Dt: 166787.79216119126
VIF Channel_Type_Cd: 861.09592836073
VIF Auto_Ship_Ind: 2.146000403097909
VIF Interior_Decor_Signage_Available_Funds: 4.942077816604081
VIF LMS_Enrolled: 7.997987022380679
VIF LMS_Available_Matching_Funds: 1.6579891343946211
VIF LMS_Used_Funds_Most_Recent_3_Months: 3.4369092181080823
VIF LMS_Used_Funds_Previous_3_Months: 3.4036959320820066
VIF Open_Smart: 1.3565776640250051
VIF PIP_Program: 2.203341013424788
VIF KYN_Training: 15581.257511349677
VIF PQS_Program: 10.848809544909837
VIF Last_EQuIPP_Login: 20634.132387700727
VIF PQS_Enrolled_Date: 158661.25631730212
VIF SCS_Program: 4.1777450040510535
VIF Spar_Coverage_Ind: 4.826557193587186
VIF Specialty_Solutions: 1.1312425850305632
VIF Vaccine_Items: 2.7339459579115784
VIF Vaccine_Starter: 1.0728232092588774
VIF Vaccine_Items_Count: 3.0667708066557813
VIF Vaccine_Items_Sls_Amt: 2.27507498587236
VIF YPO: 1.6442642908483625
VIF Tot_Sls_Amt: 276574423641.75366
VIF Brand_Rx_Sls

In [12]:
X1 = X.drop(df_clean[date_cols], axis=1)       # removing highly correlated date columns (b/c converted to numeric)
for i, col in enumerate(X1.columns):            # check VIF
    print('VIF {}: {}'.format(col,variance_inflation_factor(X1.values,i)))

VIF Zip: 17.605049977597528
VIF Channel_Type_Cd: 180.74268976755818
VIF Auto_Ship_Ind: 2.1431289568408256
VIF Interior_Decor_Signage_Available_Funds: 4.860019524719944
VIF LMS_Enrolled: 7.919643091999739
VIF LMS_Available_Matching_Funds: 1.6410297847545097
VIF LMS_Used_Funds_Most_Recent_3_Months: 3.4290422041552273
VIF LMS_Used_Funds_Previous_3_Months: 3.400218149356471
VIF Open_Smart: 1.3304060281793773
VIF PIP_Program: 2.197827495676608
VIF PQS_Program: 8.43104622272699
VIF SCS_Program: 4.149776127875367
VIF Spar_Coverage_Ind: 4.770822085858102
VIF Specialty_Solutions: 1.130739989143978
VIF Vaccine_Items: 2.7265039392678494
VIF Vaccine_Starter: 1.0701675359864176
VIF Vaccine_Items_Count: 3.0636333680915984
VIF Vaccine_Items_Sls_Amt: 2.270733087298727
VIF YPO: 1.6417811641608955
VIF Tot_Sls_Amt: 276438610770.67773
VIF Brand_Rx_Sls_Amt: 214676913381.34262
VIF Gnrc_Rx_Sls_Amt: 101858012.74042861
VIF Os_Gnrc_Rx_Sls_Amt: 5278892022.609001
VIF Otc_Sls_Amt: 367919638.95987093
VIF 340B_Activ

In [13]:
X2 = X1.drop(['Brand_Rx_Sls_Amt','Gnrc_Rx_Sls_Amt','Os_Gnrc_Rx_Sls_Amt','Otc_Sls_Amt'], axis=1)    # remove $ sub-cats

In [87]:
for i, col in enumerate(X2.columns):
    print('VIF {}: {}'.format(col,variance_inflation_factor(X2.values,i))) 

VIF Zip: 17.499720236425627
VIF Channel_Type_Cd: 178.60134539973365
VIF Auto_Ship_Ind: 2.125540980730421
VIF Interior_Decor_Signage_Available_Funds: 4.855752707559133
VIF LMS_Enrolled: 7.892263638835095
VIF LMS_Available_Matching_Funds: 1.6400277369935408
VIF LMS_Used_Funds_Most_Recent_3_Months: 3.4212271110703525
VIF LMS_Used_Funds_Previous_3_Months: 3.397397934478947
VIF Open_Smart: 1.329562952190002
VIF PIP_Program: 2.196758507380446
VIF PQS_Program: 8.425823316817128
VIF SCS_Program: 4.146574773147945
VIF Spar_Coverage_Ind: 4.760256460843783
VIF Specialty_Solutions: 1.1293134283904107
VIF Vaccine_Items: 2.7038479965265587
VIF Vaccine_Starter: 1.0698188300496838
VIF Vaccine_Items_Count: 3.056872774062806
VIF Vaccine_Items_Sls_Amt: 2.249114789383045
VIF YPO: 1.6353573425177121
VIF Tot_Sls_Amt: 1.6354117806178923
VIF 340B_Active: 95.72835472204008
VIF Profitability: 3.6904078268875145
VIF Annual_Dispense_Fee_Total: 7.545222790140893
VIF Macro_PID: 2.9024755109628453
VIF Script_Count: 

In [14]:
X3 = X2.drop(['Channel_Type_Cd', '340B_Active','Zip','Region_Name_encoded','Buying_Group_encoded','3rd_Party_Vendor_encoded'], axis=1)    # remove high VIF value

In [16]:
MH_only_cols = ['Profitability',
       'Annual_Dispense_Fee_Total', 'Macro_PID', 'Script_Count',
       'Capture_Rate', 'Specialty_Spend', 'Specialty_Product_Purchases',
       'Generic', 'Brand', 'Generic_Ratio']
X3 = X3.drop(df_clean[MH_only_cols], axis=1)

In [94]:
for i, col in enumerate(X3.columns):
    print('VIF {}: {}'.format(col,variance_inflation_factor(X3.values,i))) 

VIF Auto_Ship_Ind: 2.052200894950357
VIF Interior_Decor_Signage_Available_Funds: 4.660827427512013
VIF LMS_Enrolled: 7.088363837713384
VIF LMS_Available_Matching_Funds: 1.59470373598972
VIF LMS_Used_Funds_Most_Recent_3_Months: 3.4065931564378054
VIF LMS_Used_Funds_Previous_3_Months: 3.3776776803037087
VIF Open_Smart: 1.2651425666042655
VIF PIP_Program: 2.184338552262139
VIF PQS_Program: 7.9368326598802525
VIF SCS_Program: 4.057779920500175
VIF Spar_Coverage_Ind: 4.282448228615
VIF Specialty_Solutions: 1.1234214043629231
VIF Vaccine_Items: 2.681583676965503
VIF Vaccine_Starter: 1.066193020249581
VIF Vaccine_Items_Count: 3.051740296485936
VIF Vaccine_Items_Sls_Amt: 2.2362709807729657
VIF YPO: 1.6165183377243193
VIF Tot_Sls_Amt: 1.6065453258223077
VIF DLC_Program_encoded: 4.980035868145702
VIF FEM_Program_encoded: 3.9218684876129664
VIF HM_Circular_Program_encoded: 4.272935011493843
VIF Internal_Decor: 2.8688959159696323
VIF PQS_Enrolled_encoded: 2.067641857158895
VIF ST_encoded: 3.454607

In [103]:
# need to do test_train_split

X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.33, random_state=42)

In [104]:
from sklearn.metrics import recall_score, average_precision_score

In [105]:
# random forest

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.feature_importances_
pred_rf = rf.predict(X_test) 
rf_score_r2 = np.mean(cross_val_score(rf, X_train, y_train, scoring='r2', cv=5))
rf_score_recall = recall_score(y_test, pred_rf, average='weighted')
rf_score_avgprecision = average_precision_score(y_test, pred_rf)
print(rf_score_r2, rf_score_recall, rf_score_avgprecision)

0.291487662175 0.970625 0.248448275862


In [106]:
feature_imp = pd.DataFrame({'Portion of Samples Affected' : rf.feature_importances_}, index=X3.columns)
print(feature_imp)

                                        Portion of Samples Affected
Auto_Ship_Ind                                              0.010925
Interior_Decor_Signage_Available_Funds                     0.028809
LMS_Enrolled                                               0.006030
LMS_Available_Matching_Funds                               0.044850
LMS_Used_Funds_Most_Recent_3_Months                        0.002833
LMS_Used_Funds_Previous_3_Months                           0.005038
Open_Smart                                                 0.001660
PIP_Program                                                0.011891
PQS_Program                                                0.002252
SCS_Program                                                0.011452
Spar_Coverage_Ind                                          0.008817
Specialty_Solutions                                        0.000000
Vaccine_Items                                              0.010923
Vaccine_Starter                                 

In [107]:
# decision tree

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt.feature_importances_
pred_dt = dt.predict(X_test) 
dt_score_r2 = np.mean(cross_val_score(dt, X_train, y_train, scoring='r2', cv=5))
dt_score_recall = recall_score(y_test, pred_dt, average='weighted')
dt_score_avgprecision = average_precision_score(y_test, pred_dt)
print(dt_score_r2, dt_score_recall, dt_score_avgprecision)

0.0372847265824 0.961875 0.271782161804


In [108]:
# some way to plot pred_rf against y_test
# and plot pred_dt against y_test

In [109]:
# don't know if logit (logistic regression) has an r2?
# trying to beat an r2 of .69

In [ ]:
X3.hist
plt.tight_layout()

In [22]:
###########################################
# TEST BLOCK
###########################################

X_gbc = X3
# initial run making sure it works
X_gbc_train, X_gbc_test, y_train, y_test = train_test_split(X_gbc, y, test_size=0.33, random_state=42)

model = GradientBoostingClassifier().fit(X_gbc_train, y_train)
print(model.score(X_gbc_test, y_test))

# commented out since we already have our predictor
cv = KFold(n_splits=10)
accuracy = []
for estimator in range(100, 1000, 100):
    for depth in range(1, 4):
        for sample in (0.1, 0.3, 0.5, 0.7, 0.9):
            model = GradientBoostingClassifier(n_estimators=estimator, max_depth=depth, subsample=sample)
            scores = []
            for train_i, test_i in cv.split(X_gbc):
                # Xr, yr, Xt, yt = X.loc[train_i], y.loc[train_i], X.loc[test_i], y.loc[test_i]
                Xr, yr, Xt, yt = X.loc[train_i], y.loc[train_i], X.loc[test_i], y.loc[test_i]
                model.fit(Xr, yr)
                scores.append(model.score(Xt, yt))
                accuracy = sum(scores) / len(scores)
            print('sample:', sample, 'depth:', depth, 'estimators:', estimator, 'accuracy:', sum(scores) / len(scores))
print(accuracy.max)

# best hyperparameters
# sample: 0.5 depth: 1 estimators: 100 accuracy: 0.808817204301
# model = GradientBoostingClassifier(
#     n_estimators=100, max_depth=1, subsample=0.9).fit(X, y)

0.975625
sample: 0.1 depth: 1 estimators: 100 accuracy: 0.995461361506
sample: 0.3 depth: 1 estimators: 100 accuracy: 0.995667547073
sample: 0.5 depth: 1 estimators: 100 accuracy: 0.996905086479
sample: 0.7 depth: 1 estimators: 100 accuracy: 0.997317457613
sample: 0.9 depth: 1 estimators: 100 accuracy: 0.998350515464
sample: 0.1 depth: 2 estimators: 100 accuracy: 0.995874584647
sample: 0.3 depth: 2 estimators: 100 accuracy: 0.998350515464
sample: 0.5 depth: 2 estimators: 100 accuracy: 0.998350515464
sample: 0.7 depth: 2 estimators: 100 accuracy: 0.998350515464
sample: 0.9 depth: 2 estimators: 100 accuracy: 0.997937718327
sample: 0.1 depth: 3 estimators: 100 accuracy: 0.996080344211
sample: 0.3 depth: 3 estimators: 100 accuracy: 0.998350515464
sample: 0.5 depth: 3 estimators: 100 accuracy: 0.998350515464
sample: 0.7 depth: 3 estimators: 100 accuracy: 0.998350515464
sample: 0.9 depth: 3 estimators: 100 accuracy: 0.998144329897
sample: 0.1 depth: 1 estimators: 200 accuracy: 0.995874158644

sample: 0.7 depth: 3 estimators: 900 accuracy: 0.998350515464
sample: 0.9 depth: 3 estimators: 900 accuracy: 0.998144329897
<built-in method max of numpy.float64 object at 0x000000000C5F2300>
